In [1]:
import pandas as pd
import numpy as np

In [2]:
bakrep_metadata = pd.read_csv("data/bakrep-export_18112024.tsv",sep='\t', low_memory=False, index_col="#id")

In [3]:
[tup for tup in zip(bakrep_metadata.columns, bakrep_metadata.dtypes)]

[('bakta.genome.genus', dtype('O')),
 ('bakta.genome.species', dtype('O')),
 ('bakta.genome.strain', dtype('O')),
 ('bakta.stats.no_sequences', dtype('float64')),
 ('bakta.stats.size', dtype('float64')),
 ('bakta.stats.gc', dtype('float64')),
 ('bakta.stats.n_ratio', dtype('float64')),
 ('bakta.stats.n50', dtype('float64')),
 ('bakta.stats.coding_ratio', dtype('float64')),
 ('gtdbtk.classification.domain', dtype('O')),
 ('gtdbtk.classification.phylum', dtype('O')),
 ('gtdbtk.classification.class', dtype('O')),
 ('gtdbtk.classification.order', dtype('O')),
 ('gtdbtk.classification.family', dtype('O')),
 ('gtdbtk.classification.genus', dtype('O')),
 ('gtdbtk.classification.species', dtype('O')),
 ('mlst.sequence_type', dtype('O')),
 ('checkm2.quality.completeness', dtype('float64')),
 ('checkm2.quality.contamination', dtype('float64')),
 ('metadata.accession', dtype('O')),
 ('metadata.collected_by', dtype('O')),
 ('metadata.collection_date', dtype('O')),
 ('metadata.country', dtype('O'))

In [4]:
bakrep_metadata[["metadata.accession"]]

,metadata.accession
#id,
SAMD00000344,SAMD00000344
SAMD00000345,SAMD00000345
SAMD00000355,SAMD00000355
SAMD00000550,SAMD00000550
SAMD00000552,SAMD00000552
...,...
SAMN10405967,SAMN10405967
SAMN10405968,SAMN10405968
SAMN10407504,SAMN10407504


In [5]:
est_card = pd.read_csv('/Users/ktruong/data/661k/est_card_661k.csv', header = None, names= ['id','card'], index_col= 'id' )

In [6]:
est_card

,card
id,
SAMN02844126,4722564.00
SAMN04558116,1589918.00
SAMEA2068521,6482382.50
SAMN09281158,2937214.75
SAMN09843229,4991481.50
...,...
SAMEA3481361,2161193.00
SAMN09650548,5065128.50
SAMN09463236,5239581.00


In [7]:
# missing 3 genomes
set(est_card.index.values) - set(bakrep_metadata.index.values)

{'SAMD00080514', 'SAMD00080519', 'SAMD00080520'}

In [8]:
species_groups =  bakrep_metadata.groupby(['gtdbtk.classification.species']).groups
group_sizes = bakrep_metadata.groupby(['gtdbtk.classification.species']).size()

In [9]:
group_sizes.sort_values()

gtdbtk.classification.species
1-14-0-20-39-49 sp002787635         1
Pantoea brenneri                    1
Pantoea anthophila                  1
Pantoea allii                       1
Pannonibacter phragmitetus          1
                                ...  
Staphylococcus aureus           47997
Mycobacterium tuberculosis      49020
Streptococcus pneumoniae        51433
Escherichia coli                89136
Salmonella enterica            178585
Length: 8207, dtype: int64

In [10]:
group_sizes_small = group_sizes[group_sizes<100]
len(group_sizes_small)

8067

In [11]:
group_sizes_small.sort_values()

gtdbtk.classification.species
1-14-0-20-39-49 sp002787635        1
Paraburkholderia kururiensis       1
Paraburkholderia dilworthii        1
Paraburkholderia diazotrophica     1
Paraburkholderia azotifigens       1
                                  ..
Ligilactobacillus salivarius      96
Elizabethkingia anophelis         96
Escherichia marmotae              97
Deinococcus radiodurans           98
Chlamydophila pecorum             98
Length: 8067, dtype: int64

In [12]:
group_sizes[(group_sizes>100) & (group_sizes<1000)].sort_values()

gtdbtk.classification.species
Enterobacter kobei              104
Bacillus_A paranthracis         105
Aeromonas veronii               106
Moraxella catarrhalis           107
Salmonella bongori              107
                               ... 
Mycobacterium intracellulare    880
Mycobacterium marinum           891
Bacillus_A anthracis            893
Neisseria lactamica             951
Salmonella houtenae             972
Length: 104, dtype: int64

In [13]:
group_sizes.sort_values(ascending=False).head(20)

gtdbtk.classification.species
Salmonella enterica           178585
Escherichia coli               89136
Streptococcus pneumoniae       51433
Mycobacterium tuberculosis     49020
Staphylococcus aureus          47997
Campylobacter_D jejuni         28336
Neisseria meningitidis         17153
Streptococcus pyogenes         16821
Klebsiella pneumoniae          13637
Clostridioides difficile       13579
Listeria monocytogenes         12245
Listeria monocytogenes_B       12244
Streptococcus agalactiae       10359
Neisseria gonorrhoeae           8904
Campylobacter_D coli            8849
Enterococcus_B faecium          8284
Pseudomonas aeruginosa          6286
Vibrio cholerae                 5562
Acinetobacter baumannii         5124
Mycobacterium abscessus         2689
dtype: int64

In [14]:
species_groups_bakta =  bakrep_metadata.groupby(['bakta.genome.species']).groups
group_sizes_bakta = bakrep_metadata.groupby(['bakta.genome.species']).size()

In [15]:
group_sizes_bakta.sort_values(ascending=False).head(20)

bakta.genome.species
enterica         177020
coli              81894
pneumoniae        65831
tuberculosis      48945
aureus            48357
jejuni            27491
monocytogenes     24676
meningitidis      16641
pyogenes          16622
difficile         13466
agalactiae        10330
gonorrhoeae        8797
faecium            8429
sp.                8216
sonnei             7583
aeruginosa         6168
cholerae           5906
baumannii          5127
flexneri           4904
abscessus          2694
dtype: int64

In [16]:
species_more_than_50 = group_sizes[group_sizes>=50].index

In [17]:
species_more_than_50

Index(['Achromobacter xylosoxidans', 'Acinetobacter baumannii',
       'Acinetobacter nosocomialis', 'Acinetobacter pittii',
       'Actinobacillus lignieresii', 'Aeromonas caviae', 'Aeromonas dhakensis',
       'Aeromonas hydrophila', 'Aeromonas salmonicida', 'Aeromonas veronii',
       ...
       'Vibrio cholerae', 'Vibrio mediterranei', 'Vibrio parahaemolyticus',
       'Vibrio vulnificus', 'Wolbachia pipientis', 'Xanthomonas oryzae',
       'Xanthomonas phaseoli', 'Yersinia enterocolitica', 'Yersinia pestis',
       'Yersinia ruckeri'],
      dtype='object', name='gtdbtk.classification.species', length=204)

In [18]:
import random

In [19]:
dict_sample_50 = {key:random.sample(list(species_groups[key].values),50) for key in species_more_than_50}

In [20]:
dict_sample_50

{'Achromobacter xylosoxidans': ['SAMEA2247581',
  'SAMN03480300',
  'SAMN05735321',
  'SAMN03225278',
  'SAMN03753629',
  'SAMN03753632',
  'SAMEA2247585',
  'SAMN03753628',
  'SAMEA2258935',
  'SAMEA2247571',
  'SAMEA2258938',
  'SAMEA2247577',
  'SAMN03996294',
  'SAMEA2247574',
  'SAMN02599179',
  'SAMN04480366',
  'SAMN06265358',
  'SAMN03604112',
  'SAMN04477589',
  'SAMEA2258936',
  'SAMN03753631',
  'SAMN03436202',
  'SAMEA3710557',
  'SAMN03475389',
  'SAMEA2247576',
  'SAMN03753627',
  'SAMN03892125',
  'SAMEA2247587',
  'SAMEA20230168',
  'SAMN03197818',
  'SAMEA2247588',
  'SAMEA2247595',
  'SAMEA102775168',
  'SAMEA102775918',
  'SAMN03197817',
  'SAMEA2247589',
  'SAMD00013333',
  'SAMEA1982604',
  'SAMN03753630',
  'SAMEA104305326',
  'SAMN04480442',
  'SAMEA4547755',
  'SAMEA2258937',
  'SAMEA2175157',
  'SAMEA2247583',
  'SAMEA3357506',
  'SAMD00049594',
  'SAMN04875581',
  'SAMN06014870',
  'SAMEA2247584'],
 'Acinetobacter baumannii': ['SAMN05238693',
  'SAMD00030181',

In [24]:
# Function to generate n groups with maximum diversity and a specified number of values per group
def generate_diverse_groups(data_dict, n, values_per_group):
    # Step 1: Flatten the dictionary into a list of (key, value) pairs
    flattened = [(key, value) for key, values in data_dict.items() for value in values]
    
    # Step 2: Shuffle the list randomly
    random.shuffle(flattened)
    
    # Step 3: Initialize the groups with empty lists and a dictionary to track key counts per group
    groups = [[] for _ in range(n)]
    group_keys = [set() for _ in range(n)]  # Tracks the keys in each group
    
    # Step 4: Distribute values into groups while keeping diversity
    for i, (key, value) in enumerate(flattened):
        # Try to find a group that doesn't already have a value from this key
        added = False
        
        # First, try to keep diversity as high as possible
        for group_index in range(n):
            if len(groups[group_index]) < values_per_group and key not in group_keys[group_index]:
                groups[group_index].append(value)
                group_keys[group_index].add(key)
                added = True
                break
        
        # If no group can be found with strict diversity, relax the constraint and add the value
        if not added:
            for group_index in range(n):
                if len(groups[group_index]) < values_per_group:
                    groups[group_index].append(value)
                    group_keys[group_index].add(key)  # Even if key is repeated, still add to track it
                    break
    
    # Step 5: Prepare the results by removing the keys from the tuples
    groups = [[value for value in group] for group in groups]

    # Step 6: Calculate and return the number of keys in each group
    key_counts = [len(keys) for keys in group_keys]

    return groups, key_counts

In [25]:
n = 100  # Number of groups to create
v = 50
groups,species_count = generate_diverse_groups(dict_sample_50, n, v)

In [26]:
[print(g) for g in groups]

['SAMN03276104', 'SAMEA1572088', 'SAMN05391707', 'SAMN06314545', 'SAMEA1411784', 'SAMEA1929603', 'SAMN07333106', 'SAMN08467231', 'SAMN03449466', 'SAMN02797514', 'SAMEA4558931', 'SAMN08458218', 'SAMEA103924441', 'SAMN03455365', 'SAMEA2247577', 'SAMN06892246', 'SAMN05898574', 'SAMEA2538972', 'SAMN02356609', 'SAMEA104027824', 'SAMN07421921', 'SAMN09726931', 'SAMEA104208677', 'SAMN07660233', 'SAMEA102394168', 'SAMN07452575', 'SAMN08438789', 'SAMEA3719179', 'SAMN08362516', 'SAMD00017005', 'SAMEA4483244', 'SAMN03002082', 'SAMN03948580', 'SAMN03284613', 'SAMN04448495', 'SAMEA3391282', 'SAMEA767921', 'SAMEA104148129', 'SAMN08093265', 'SAMEA3481549', 'SAMEA80878918', 'SAMN07613268', 'SAMN06909340', 'SAMN02581394', 'SAMEA1920399', 'SAMN03074840', 'SAMD00006169', 'SAMN08611276', 'SAMN09831753', 'SAMN07660288']
['SAMN06314534', 'SAMEA4051822', 'SAMN06076989', 'SAMN08816058', 'SAMN03753631', 'SAMN01090325', 'SAMN07452545', 'SAMEA751500', 'SAMN09385296', 'SAMEA91498918', 'SAMEA2040658', 'SAMEA346822

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [27]:
def write_list_to_file(filename, values):
    """
    Write a list of values to a text file, each value on a new line.
    
    :param filename: Path to the output file.
    :param values: List of values to write.
    """
    with open(filename, "w") as f:
        for value in values:
            f.write(f"{value}\n")

In [29]:
for idx, values in enumerate(groups):
    path = 'data/sample_100_batches_50_gen/'
    if (idx+1)<10:
        write_list_to_file(path+f'sample_0{idx+1}.txt',values)
    else:
        write_list_to_file(path+f'sample_{idx+1}.txt',values)